In [26]:
from math import exp

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [ ]:
_train_=pd.read_csv('../input/train.csv')
_train_.describe()
_train_=_train_[_train_['accuracy']>5]


places=_train_['place_id'].unique()

train=_train_[:int(len(_train_)*0.9)]
test=_train_[int(len(_train_)*0.9):]



train.loc[:,'day']=train.loc[:, 'time'].div(1440).map(int)
train.loc[:,'weekday']=train.loc[:,'day']%7
train.loc[:,'year']=train.loc[:,'day'].div(365).map(int)
train.loc[:,'month']=(train.loc[:,'day']%365).div(30).map(int)
train.loc[:,'hour']=train.loc[:,'time'].div(60).map(int)
train.loc[:, 'hourInDay']=train.loc[:,'hour']%24


test.loc[:, 'day']=test.loc[:,'time'].div(1440).map(int)
test.loc[:, 'weekday']=test.loc[:, 'day']%7
test.loc[:,'year']=test.loc[:,'day'].div(365).map(int)
test.loc[:,'month']=(test.loc[:,'day']%365).div(30).map(int)
test.loc[:, 'hour']=test.loc[:, 'time'].div(60).map(int)
test.loc[:, 'hourInDay']=test.loc[:,'hour']%24

In [ ]:
g=train.groupby('place_id')
summary=pd.DataFrame()
summary['count']=g['y'].count()

summary['yMin']=g['y'].min()
summary['yMax']=g['y'].max()
summary['yMean']=g['y'].mean()
summary['yStd']=g['y'].std()

summary['xMin']=g['x'].min()
summary['xMax']=g['x'].max()
summary['xMean']=g['x'].mean()
summary['xStd']=g['x'].std()

summary['dayMin']=g['day'].min()
summary['dayMax']=g['day'].max()
summary['dayMean']=g['day'].mean()
summary['dayStd']=g['day'].std()

summary['weekdayMean']=g['weekday'].mean()
summary['weekdayStd']=g['weekday'].std()

summary['monthMean']=g['month'].mean()
summary['monthStd']=g['month'].std()

summary.sort_values(by='yMin', inplace=True, ascending=False)
summary.head(5)

f=['x','y', 'weekday', 'day', 'hourInDay']



In [ ]:
nClasses=2000
place_ids=set(summary.index[:nClasses])
place_list=summary.index[:nClasses]

subTrain=train[train['place_id'].isin(place_ids)]
subTest=test[test['place_id'].isin(place_ids)]

In [ ]:
n_neighbors=2000
weights=lambda x: 1/x
knnClf=KNeighborsClassifier(n_neighbors, weights)
knnClf.fit(subTrain[f], subTrain['place_id'])
nbrClasses=knnClf.predict_proba(subTest[f])
nbrClasses=[sorted([(1.0*p, knnClf.classes_[i]) for i,p in enumerate(nbrs) if p>0])[::-1] for nbrs in nbrClasses]
knnPred=[[n[1] for n in nbrs] for nbrs in nbrClasses]

In [ ]:
evaluation=[subTest['place_id'].iloc[i] in knnPred[i] for i in xrange(len(subTest))]
1.0*sum(evaluation)/len(evaluation)
score=0
# have to use in topPlaces, because some itmes in topPlaces are empty.
1.0*sum([subTest['place_id'].iloc[i] in knnPred[i][:1] for i in xrange(len(subTest))])/len(evaluation)

1.0*sum([subTest['place_id'].iloc[i] in knnPred[i][:3] for i in xrange(len(subTest))])/len(evaluation)

score+=sum([subTest['place_id'].iloc[i] in knnPred[i][:1] for i in xrange(len(subTest))])
score+=0.5*sum([subTest['place_id'].iloc[i] in knnPred[i][1:2] for i in xrange(len(subTest))])
score+=0.33*sum([subTest['place_id'].iloc[i] in knnPred[i][2:3] for i in xrange(len(subTest))])

1.0*score/len(evaluation)